### **Download Dataset**

In [1]:
# https://drive.google.com/file/d/1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV/view
!gdown 1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV

Downloading...
From (original): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV
From (redirected): https://drive.google.com/uc?id=1kUy2tuH-kKBlFCNA0a9sqD2TG4uyvBnV&confirm=t&uuid=ded79385-9c07-4387-978a-4f51a22765fe
To: d:\Scene Text Recognition\icdar2003.zip

  0%|          | 0.00/45.7M [00:00<?, ?B/s]
  1%|          | 524k/45.7M [00:00<00:29, 1.51MB/s]
  2%|▏         | 1.05M/45.7M [00:00<00:20, 2.17MB/s]
  3%|▎         | 1.57M/45.7M [00:00<00:17, 2.59MB/s]
  5%|▍         | 2.10M/45.7M [00:00<00:14, 2.91MB/s]
  6%|▌         | 2.62M/45.7M [00:00<00:14, 3.07MB/s]
  7%|▋         | 3.15M/45.7M [00:01<00:13, 3.22MB/s]
  8%|▊         | 3.67M/45.7M [00:01<00:12, 3.27MB/s]
  9%|▉         | 4.19M/45.7M [00:01<00:12, 3.31MB/s]
 10%|█         | 4.72M/45.7M [00:01<00:12, 3.34MB/s]
 11%|█▏        | 5.24M/45.7M [00:01<00:12, 3.28MB/s]
 13%|█▎        | 5.77M/45.7M [00:01<00:11, 3.40MB/s]
 14%|█▍        | 6.29M/45.7M [00:02<00:11, 3.47MB/s]
 15%|█▍        | 6.82M/45.7M [00:02<00:11, 

In [3]:
import zipfile 
import os 

with zipfile.ZipFile("icdar2003.zip", 'r') as zip_ref:
    zip_ref.extractall("datasets")

print("Done extracting.")

Done extracting.


### **Prepare Dataset**

In [5]:
import xml.etree.ElementTree as ET 

words_xml_path = 'datasets/SceneTrialTrain/words.xml'
tree = ET.parse(words_xml_path)
root = tree.getroot()

print(root)
for image in root:
    print(image)

<Element 'tagset' at 0x000001E313D0FF60>
<Element 'image' at 0x000001E313D0FE70>
<Element 'image' at 0x000001E313D29030>
<Element 'image' at 0x000001E313D28FE0>
<Element 'image' at 0x000001E313D29D00>
<Element 'image' at 0x000001E313D2AF70>
<Element 'image' at 0x000001E313D2B880>
<Element 'image' at 0x000001E313D301D0>
<Element 'image' at 0x000001E313D30EA0>
<Element 'image' at 0x000001E313D31E40>
<Element 'image' at 0x000001E313D32660>
<Element 'image' at 0x000001E313D33600>
<Element 'image' at 0x000001E313D33A60>
<Element 'image' at 0x000001E313D484A0>
<Element 'image' at 0x000001E313D489F0>
<Element 'image' at 0x000001E313D48F40>
<Element 'image' at 0x000001E313D491C0>
<Element 'image' at 0x000001E313D4A070>
<Element 'image' at 0x000001E313D4A2F0>
<Element 'image' at 0x000001E313D4ACF0>
<Element 'image' at 0x000001E313D4B6F0>
<Element 'image' at 0x000001E313D60040>
<Element 'image' at 0x000001E313D60590>
<Element 'image' at 0x000001E313D61260>
<Element 'image' at 0x000001E313D63B50>

In [6]:
for image in root:
    print(image[0].text)
    print(image[1].attrib['x'], image[1].attrib['y'])
    break

apanar_06.08.2002/IMG_1261.JPG
1600 1200


In [7]:
for image in root:
    img_name = image[0].text
    for bbs in image.findall('taggedRectangles'):
        for bb in bbs:
            bbox = [ 
                bb.attrib['x'], 
                bb.attrib['y'],
                bb.attrib['width'], 
                bb.attrib['height']
            ]
            print(bb[0].text, bbox)
    break 

self ['174.0', '392.0', '274.0', '195.0']
adhesive ['512.0', '391.0', '679.0', '183.0']
address ['184.0', '612.0', '622.0', '174.0']
labels ['863.0', '599.0', '446.0', '187.0']
36 ['72.0', '6.0', '95.0', '87.0']
89m ['247.0', '2.0', '197.0', '88.0']
cls ['792.0', '0.0', '115.0', '81.0']
250 ['200.0', '848.0', '228.0', '139.0']
on ['473.0', '878.0', '165.0', '109.0']
a ['684.0', '878.0', '71.0', '106.0']
roll ['806.0', '844.0', '218.0', '141.0']


In [8]:
def extract_data_from_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()

    image_paths = []
    image_sizes = []
    image_labels = []
    bounding_boxes = []

    for image in root:
        bbs_of_image = []
        labels_of_image = []

        for bbs in image.findall("taggedRectangles"):
            for bb in bbs:
                if not bb[0].text.isalnum():
                    continue 

                if "é" in bb[0].text.lower() or "ñ" in bb[0].text.lower():
                    continue 

                bbs_of_image.append( 
                    [ 
                        float(bb.attrib["x"]), 
                        float(bb.attrib["y"]), 
                        float(bb.attrib["width"]), 
                        float(bb.attrib["height"]),
                    ]
                )
                labels_of_image.append(bb[0].text.lower())
        image_paths.append(image[0].text)
        image_sizes.append((int(image[1].attrib["x"]), int(image[1].attrib["y"])))
        bounding_boxes.append(bbs_of_image)
        image_labels.append(labels_of_image)
    return image_paths, image_sizes, image_labels, bounding_boxes 

image_paths, image_sizes, image_labels, bounding_boxes = extract_data_from_xml(words_xml_path)

In [9]:
for path, size, label, bbox in zip(image_paths, image_sizes, image_labels, bounding_boxes):
    print(path, size, label, bbox)
    break

apanar_06.08.2002/IMG_1261.JPG (1600, 1200) ['self', 'adhesive', 'address', 'labels', '36', '89m', 'cls', '250', 'on', 'a', 'roll'] [[174.0, 392.0, 274.0, 195.0], [512.0, 391.0, 679.0, 183.0], [184.0, 612.0, 622.0, 174.0], [863.0, 599.0, 446.0, 187.0], [72.0, 6.0, 95.0, 87.0], [247.0, 2.0, 197.0, 88.0], [792.0, 0.0, 115.0, 81.0], [200.0, 848.0, 228.0, 139.0], [473.0, 878.0, 165.0, 109.0], [684.0, 878.0, 71.0, 106.0], [806.0, 844.0, 218.0, 141.0]]
